In [3]:
'''
    NOTEBOOK QUE TEM COMO OBJETIVO MOSTRAR O BASICO DE TRATAMENTO DE DADOS COM O DATAFRAME, PRINCIPAL MÉTODO DE MANIPULAÇÃO DE DADOS 
COM PYTHON E SPARK.
'''

# Exemploes de data frames com spark 

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exemplo DataFrame").getOrCreate() # inicializa o spark


df = spark.createDataFrame([("João", 10),("Maria", 20),("Pedro", 30)])
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/14 21:46:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+-----+---+
|   _1| _2|
+-----+---+
| João| 10|
|Maria| 20|
|Pedro| 30|
+-----+---+



In [3]:
# Exemplo DF com schema 

schema = "ID int, Nome string"
dados = [[1, "João"], [2, "Maria"]]

df2 = spark.createDataFrame(dados, schema)
df2.show()


+---+-----+
| ID| Nome|
+---+-----+
|  1| João|
|  2|Maria|
+---+-----+



In [6]:
# Exemplo de operação sum no data frame
from pyspark.sql.functions import sum

schema2 = "produto string, venda int"
dados = [["caneta", 10],["lapis", 10],["caneta", 40]]

df3 = spark.createDataFrame(dados, schema2)
df3.show()

agrupado = df3.groupBy("produto").agg(sum("venda"))
agrupado.show()


+-------+-----+
|produto|venda|
+-------+-----+
| caneta|   10|
|  lapis|   10|
| caneta|   40|
+-------+-----+

+-------+----------+
|produto|sum(venda)|
+-------+----------+
| caneta|        50|
|  lapis|        10|
+-------+----------+



In [7]:
# Exemplo select para seleção de colunas quando necessário

df3.select("venda", "produto").show()

+-----+-------+
|venda|produto|
+-----+-------+
|   10| caneta|
|   10|  lapis|
|   40| caneta|
+-----+-------+



In [9]:
# Exemplo de expressões 
from pyspark.sql.functions import expr

df3.select("venda", "produto", expr("venda * 2")).show()

+-----+-------+-----------+
|venda|produto|(venda * 2)|
+-----+-------+-----------+
|   10| caneta|         20|
|   10|  lapis|         20|
|   40| caneta|         80|
+-----+-------+-----------+



In [5]:
# Carregando dados para um DataFrame utilizando read.csv
from pyspark.sql.types import *

arqschema = " id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"

despachantes = spark.read.csv("despachantes.csv", header = False, schema = arqschema)
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [6]:
# Carregando os mesmos dados utilizando read.load

desp_autoschema = spark.read.load("despachantes.csv", header = False, format="csv", sep= ",", inferSchema = True)
desp_autoschema.show()

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+



In [11]:
# Aplicando condições com .func no dataframe

from pyspark.sql import functions as Func
# despachantes.select("id", "nome", "vendas").where(Func.col("vendas") > 20).show()

despachantes.select("id", "nome", "vendas").where((Func.col("vendas") > 20) & (Func.col("vendas") < 40)).show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
+---+-------------------+------+



In [ ]:
# Alterando nome de coluna 

newDf = despachantes.withColumnRenamed("nome", "nomes")
